In [1]:
import re,os
import numpy as np
import random
mypath="authors"

In [2]:
def dataSet(dirName):
    authors=[]
    path=os.getcwd()+'/'+dirName+'/'
    mylist=[]
    for filename in os.listdir(path):
        authors.append(filename)
        for file in os.listdir(path+filename):
            files=open(path+filename+'/'+file,'r',encoding="utf-8")
            text=files.read().split()
            mylist.append(text) 
    return mylist,authors

In [3]:
dataset,authors=dataSet(mypath)

dataset=[[w.lower() for w in line] for line in dataset] #lowercase


In [72]:
docData=[]
docAuthors=[]
defaultDocWord=530
for i in range(len(authors)):
    lenBook=len(dataset[i])
    lenDoc=int(lenBook/defaultDocWord)
    if(lenBook%defaultDocWord!=0):
        lenDoc=lenDoc+1
    for k in range(lenDoc):
        docData.append(dataset[i][k*defaultDocWord:(k+1)*defaultDocWord])
        docAuthors.append(authors[i]) 


In [5]:

def divideDataSet(docData,docAuthors):
   
    suffl = list(zip(docData, docAuthors))
    random.shuffle(suffl)
    docData, docAuthors = zip(*suffl)

    lenTrainSet=int(len(docData)*0.8)
    
    trainSetData=docData[0:lenTrainSet]
    trainSetAuthors=docAuthors[0:lenTrainSet]
    
    testSetData=docData[lenTrainSet:len(docData)]
    testSetAuthors=docAuthors[lenTrainSet:len(docAuthors)]
    return trainSetData,trainSetAuthors,testSetData,testSetAuthors


In [6]:
trainSetData,trainSetAuthors,testSetData,testSetAuthors = divideDataSet(docData,docAuthors)

In [7]:
def joinWord():
    authSet=[]
    for auth in authors:
        index=[]
        wordSet=[]
        index = [index for index, value in enumerate(trainSetAuthors) if value == auth]
        for i in index:
            wordSet.extend(trainSetData[i])
        authSet.append(wordSet)
    return authSet

In [58]:
#Distinct Word joining
authSet=joinWord()
auth_word=[list(counter(authSet[i]).items()) for i in range(len(authors))]

('noksanlığından', 1)


In [74]:
auth_word[0]

Andres platonov


In [75]:
from collections import Counter
def counter(docData):
    wordcount = Counter(docData)
    return wordcount